# Importing the required modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy

# Reading the dataset

In [51]:
curr = pd.read_csv("../data/curriculum.csv")

# Glimpse into the dataset

In [38]:
curr.head()

Courses             Topic  \
0                               Python for Everybody          Intro CS   
1  Introduction to Computer Science and Programmi...          Intro CS   
2                          How to Code - Simple Data  Core Programming   
3                         How to Code - Complex Data  Core Programming   
4                      Programming Languages, Part A  Core Programming   

   Duration           Effort  Total Hours Lower Bound  Start Date  \
0  10 weeks    10 hours/week                      100  28-07-2022   
1   9 weeks    15 hours/week                      135  01-09-2022   
2   7 weeks  8-10 hours/week                       56  18-10-2022   
3   6 weeks  8-10 hours/week                       48  06-11-2022   
4   5 weeks   4-8 hours/week                       20  23-11-2022   

  End Date Estimate Lower Bound  Total Hours Upper Bound Start Date.1  \
0                    01-09-2022                      100   28-07-2022   
1                    18-10-2022                      135   01-09-2022   
2                    06-11-2022                       70   18-10-2022   
3                    23-11-2022                       60   11-11-2022   
4                    30-11-2022                       40   02-12-2022   

  End Date Estimate Upper Bound  Actual End Date  \
0                    01-09-2022              NaN   
1                    18-10-2022              NaN   
2                    11-11-2022              NaN   
3                    02-12-2022              NaN   
4                    16-12-2022              NaN   

                   Prerequisites  
0                              -  
1            high school algebra  
2                              -  
3      How to Code - Simple Data  
4  How to Code (Hear instructor)

In [39]:
curr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Courses                        43 non-null     object 
 1   Topic                          43 non-null     object 
 2   Duration                       42 non-null     object 
 3   Effort                         42 non-null     object 
 4   Total Hours Lower Bound        43 non-null     int64  
 5   Start Date                     43 non-null     object 
 6   End Date Estimate Lower Bound  43 non-null     object 
 7   Total Hours Upper Bound        43 non-null     int64  
 8   Start Date.1                   43 non-null     object 
 9   End Date Estimate Upper Bound  43 non-null     object 
 10  Actual End Date                0 non-null      float64
 11  Prerequisites                  42 non-null     object 
dtypes: float64(1), int64(2), object(9)
memory usage: 4.2

# Alloting course codes to each course

In [5]:
def get_course_code(row):
    topic = row[1].title()
    topic = "".join(re.findall("[A-Z]",topic))
    courses_in_topic = list(curr[curr["Topic"] == row[1]]["Courses"])
    course_code = str(courses_in_topic.index(row[0]) + 1).rjust(3,"0")
    return topic + course_code

In [6]:
curr["Course Code"] = curr.apply(get_course_code,axis=1)

# Handling Course Pre-requisites

## Finding most relevant courses for prequisites using Cosine Similiarity

In [7]:
df = pd.DataFrame(columns=["Prerequisite"] + list(curr["Courses"]) + list(curr["Topic"]))

In [8]:
df

Empty DataFrame
Columns: [Prerequisite, Python for Everybody, Introduction to Computer Science and Programming using Python, How to Code - Simple Data, How to Code - Complex Data, Programming Languages, Part A, Programming Languages, Part B, Programming Languages, Part C, Object-Oriented Design, Design Patterns, Software Architecture, Calculus 1A: Differentiation, Calculus 1B: Integration, Calculus 1C: Coordinate Systems & Infinite Series, Mathematics for Computer Science, The Missing Semester of Your CS Education, Build a Modern Computer from First Principles: From Nand to Tetris, Build a Modern Computer from First Principles: Nand to Tetris Part II, Operating Systems: Three Easy Pieces, Computer Networking: a Top-Down Approach, Divide and Conquer, Sorting and Searching, and Randomized Algorithms, Graph Search, Shortest Paths, and Data Structures, Greedy Algorithms, Minimum Spanning Trees, and Dynamic Programming, Shortest Paths Revisited, NP-Complete Problems and What To Do About Them, Cybersecurity Fundamentals, Principles of Secure Coding, Identifying Security Vulnerabilities, Identifying Security Vulnerabilities in C/C++ Programming or Exploiting and Securing Vulnerabilities in Java Applications, Databases: Modeling and Theory, Databases: Relational Databases and SQL, Databases: Semistructured Data, Machine Learning, Computer Graphics, Software Engineering: Introduction, Ethics, Technology and Engineering, Intellectual Property Law in Digital Age, Data Privacy Fundamentals, Parallel Programming, Compilers, Introduction to Haskell, Learn Prolog Now!, Software Debugging, Software Testing, Final Project, Intro CS, Intro CS, Core Programming, Core Programming, Core Programming, Core Programming, Core Programming, Core Programming, Core Programming, Core Programming, Core Math, Core Math, Core Math, Core Math, CS Tools, Core Systems, Core Systems, Core Systems, Core Systems, Core Theory, Core Theory, Core Theory, Core Theory, Core Security, Core Security, Core Security, Core Security, Core Applications, Core Applications, Core Applications, Core Applications, Core Applications, Core Applications, Core Ethics, Core Ethics, Core Ethics, Advanced Programming, Advanced Programming, Advanced Programming, Advanced Programming, Advanced Programming, Advanced Programming, Final Project]
Index: []

[0 rows x 87 columns]

In [9]:
nlp = spacy.load("en_core_web_sm")
for i in set(curr["Prerequisites"]): 
    if str(type(i)) != "<class 'float'>":
        for k in i.split(";"):
            prereq = nlp(i.strip())
            row = {"Prerequisite":k}
            if i in df["Prerequisite"]:
                continue
            for j in list(curr["Courses"]) + list(curr["Topic"]):
                course = nlp(j.strip())
                row[j] = prereq.similarity(course)
            df = df.append(row,ignore_index=True)

C:\Users\mitug\AppData\Local\Temp/ipykernel_1720/914963020.py:11: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  row[j] = prereq.similarity(course)


In [10]:
#df = df.set_index("Prerequisite")

In [11]:
df

Prerequisite  Python for Everybody  \
0                                  Java for Everybody              0.887163   
1                                   C++ for Everybody              0.621831   
2                                  Java for Everybody              0.621831   
3                                      Linear Algebra              0.621831   
4                                Python for Everybody              1.000000   
5                            How to Code: Simple Data              0.347886   
6                                                   -             -0.016595   
7                          From Nand to Tetris Part I              0.658241   
8                                 high school algebra              0.369998   
9                    Mathematics for Computer Science              0.664016   
10  Graph Search, Shortest Paths, and Data Structures              0.199758   
11                                        Calculus 1C              0.267147   
12                                     Linear Algebra              0.243723   
13                                        Probability              0.243723   
14                                           basic CS              0.243723   
15                      How to Code (Hear instructor)              0.205078   
16                                   Core Programming              0.374369   
17                                            Project              0.374369   
18                      Programming Languages, Part A              0.222033   
19                                     Linear Algebra              0.399782   
20                                        Calculus 1B              0.249494   
21                      Programming Languages, Part B              0.178415   
22                             Object-Oriented Design              0.180903   
23                               Python for Everybody              0.605964   
24                             Object-Oriented Design              0.605964   
25  Greedy Algorithms, Minimum Spanning Trees, and...              0.317837   
26                                      C Programming              0.393818   
27                                   high school math              0.403973   
28  Divide and Conquer, Sorting and Searching, and...              0.273155   
29                                    Design Patterns              0.287245   
30                                        Calculus 1A              0.231365   
31                                   Core Programming              0.414884   
32                                  Scala Programming              0.479793   

    Introduction to Computer Science and Programming using Python  \
0                                            0.527122               
1                                            0.449858               
2                                            0.449858               
3                                            0.449858               
4                                            0.614067               
5                                            0.349117               
6                                           -0.165618               
7                                            0.709123               
8                                            0.367067               
9                                            0.763913               
10                                           0.439431               
11                                           0.336377               
12                                           0.246527               
13                                           0.246527               
14                                           0.246527               
15                                           0.322210               
16                                           0.440193               
17                                           0.440193               
18                                         

In [12]:
def get_most_relevant_course(row):
    best_similarity = max(row[1:])
    cols = list(df.columns)
    i = list(row[1:]).index(best_similarity)
    #print(row[0],"\t",cols[i + 1],"\t",best_similarity)
    #print(row.index,"\n")
    if best_similarity > 0.74:
        return {"Pre":row[0],"Rel":cols[i + 1],"simi":best_similarity,"incorrect":None}
    else:
        return {"Pre":row[0],"Rel":None,"simi":best_similarity,"incorrect":cols[i+1]}

In [13]:
d = pd.DataFrame(columns=["Pre","Rel","simi","incorrect"])

In [14]:
for i in df.apply(get_most_relevant_course,axis=1):
    d = d.append(i, ignore_index=True)

In [15]:
d

Pre  \
0                                  Java for Everybody   
1                                   C++ for Everybody   
2                                  Java for Everybody   
3                                      Linear Algebra   
4                                Python for Everybody   
5                            How to Code: Simple Data   
6                                                   -   
7                          From Nand to Tetris Part I   
8                                 high school algebra   
9                    Mathematics for Computer Science   
10  Graph Search, Shortest Paths, and Data Structures   
11                                        Calculus 1C   
12                                     Linear Algebra   
13                                        Probability   
14                                           basic CS   
15                      How to Code (Hear instructor)   
16                                   Core Programming   
17                                            Project   
18                      Programming Languages, Part A   
19                                     Linear Algebra   
20                                        Calculus 1B   
21                      Programming Languages, Part B   
22                             Object-Oriented Design   
23                               Python for Everybody   
24                             Object-Oriented Design   
25  Greedy Algorithms, Minimum Spanning Trees, and...   
26                                      C Programming   
27                                   high school math   
28  Divide and Conquer, Sorting and Searching, and...   
29                                    Design Patterns   
30                                        Calculus 1A   
31                                   Core Programming   
32                                  Scala Programming   

                                                  Rel      simi  \
0                                Python for Everybody  0.887163   
1                                                None  0.626999   
2                                                None  0.626999   
3                                                None  0.626999   
4                                Python for Everybody  1.000000   
5                           How to Code - Simple Data  0.747667   
6                                                None  0.228649   
7   Build a Modern Computer from First Principles:...  0.829135   
8                                                None  0.601529   
9                    Mathematics for Computer Science  1.000000   
10  Graph Search, Shortest Paths, and Data Structures  1.000000   
11                       Calculus 1A: Differentiation  0.743040   
12                                               None  0.736868   
13                                               None  0.736868   
14                                               None  0.736868   
15                                               None  0.644563   
16                 Software Engineering: Introduction  0.798630   
17                 Software Engineering: Introduction  0.798630   
18                      Programming Languages, Part A  1.000000   
19                              Software Architecture  0.787254   
20                           Calculus 1B: Integration  0.761860   
21                      Programming Languages, Part B  1.000000   
22                             Object-Oriented Design  1.000000   
23                             Object-Oriented Design  0.743627   
24                             Object-Oriented Design  0.743627   
25  Greedy Algorithms, Minimum Spanning Trees, and...  1.000000   
26                                   Core Programming  0.872170   
27                                               None  0.670473   
28  Divide and Conquer, Sorting and Searching, and...  1.000000   
29                                    Design Patterns  1.000000   
30                       Calculus 

## Adding Prerequisites to the catalog

In [16]:
def add_prereq(row):
    if row[1] == None and row[0] != "-":
        return {"Courses" : row[0].title(),"Topic":"Prerequisites"}
    return None

In [17]:
for i in d.apply(add_prereq,axis=1):
    if i != None:
        curr = curr.append(i,ignore_index=True)

In [18]:
curr = curr[["Courses","Topic","Duration","Effort"]]

In [19]:
curr.head()

Courses             Topic  \
0                               Python for Everybody          Intro CS   
1  Introduction to Computer Science and Programmi...          Intro CS   
2                          How to Code - Simple Data  Core Programming   
3                         How to Code - Complex Data  Core Programming   
4                      Programming Languages, Part A  Core Programming   

   Duration           Effort  
0  10 weeks    10 hours/week  
1   9 weeks    15 hours/week  
2   7 weeks  8-10 hours/week  
3   6 weeks  8-10 hours/week  
4   5 weeks   4-8 hours/week

In [20]:
curr.to_csv("../data/curr.csv")

In [40]:
curr = pd.read_csv("../data/curr.csv")

In [41]:
curr = curr.drop(["Unnamed: 0"],axis=1)

In [42]:
curr

Courses                 Topic  \
0                                Python for Everybody              Intro CS   
1   Introduction to Computer Science and Programmi...              Intro CS   
2                           How to Code - Simple Data      Core Programming   
3                          How to Code - Complex Data      Core Programming   
4                       Programming Languages, Part A      Core Programming   
5                       Programming Languages, Part B      Core Programming   
6                       Programming Languages, Part C      Core Programming   
7                              Object-Oriented Design      Core Programming   
8                                     Design Patterns      Core Programming   
9                               Software Architecture      Core Programming   
10                       Calculus 1A: Differentiation             Core Math   
11                           Calculus 1B: Integration             Core Math   
12  Calculus 1C: Coordinate Systems & Infinite Series             Core Math   
13                   Mathematics for Computer Science             Core Math   
14          The Missing Semester of Your CS Education              CS Tools   
15  Build a Modern Computer from First Principles:...          Core Systems   
16  Build a Modern Computer from First Principles:...          Core Systems   
17               Operating Systems: Three Easy Pieces          Core Systems   
18           Computer Networking: a Top-Down Approach          Core Systems   
19  Divide and Conquer, Sorting and Searching, and...           Core Theory   
20  Graph Search, Shortest Paths, and Data Structures           Core Theory   
21  Greedy Algorithms, Minimum Spanning Trees, and...           Core Theory   
22  Shortest Paths Revisited, NP-Complete Problems...           Core Theory   
23                         Cybersecurity Fundamentals         Core Security   
24                        Principles of Secure Coding         Core Security   
25               Identifying Security Vulnerabilities         Core Security   
26  Identifying Security Vulnerabilities in C/C++ ...         Core Security   
27                     Databases: Modeling and Theory     Core Applications   
28            Databases: Relational Databases and SQL     Core Applications   
29                     Databases: Semistructured Data     Core Applications   
30                                   Machine Learning     Core Applications   
31                                  Computer Graphics     Core Applications   
32                 Software Engineering: Introduction     Core Applications   
33                 Ethics, Technology and Engineering           Core Ethics   
34           Intellectual Property Law in Digital Age           Core Ethics   
35                          Data Privacy Fundamentals           Core Ethics   
36                               Parallel Programming  Advanced Programming   
37                                          Compilers  Advanced Programming   
38                            Introduction to Haskell  Advanced Programming   
39                                  Learn Prolog Now!  Advanced Programming   
40                                 Software Debugging  Advanced Programming   
41                                   Software Testing  Advanced Programming   
42                                      Final Project         Final Project   
43                                  C++ For Everybody         Prerequisites   
44                                 Java For Everybody         Prerequisites   
45                                     Linear Algebra         Prerequisites   
46                                High School Algebra         Prerequisites   
47                                        Probability         Prerequisites   
48                                           Basic Cs         Prerequisites   
49                      How To Code (Hear Instructor)         Prerequisites   
50               

In [43]:
def prereq2coursecodes(course : str):
    s = list(c[c["Courses"] == course]["Prerequisites"])
    if len(s) > 0 or s != ["-"]:
        return str(s[0]).split(";")
    return None

In [44]:
curr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Courses      53 non-null     object
 1   Topic        53 non-null     object
 2   Duration     52 non-null     object
 3   Effort       42 non-null     object
 4   Course Code  53 non-null     object
dtypes: object(5)
memory usage: 2.2+ KB


In [45]:
c = pd.read_csv("../data/curriculum.csv")

In [46]:
c

Courses                 Topic  \
0                                Python for Everybody              Intro CS   
1   Introduction to Computer Science and Programmi...              Intro CS   
2                           How to Code - Simple Data      Core Programming   
3                          How to Code - Complex Data      Core Programming   
4                       Programming Languages, Part A      Core Programming   
5                       Programming Languages, Part B      Core Programming   
6                       Programming Languages, Part C      Core Programming   
7                              Object-Oriented Design      Core Programming   
8                                     Design Patterns      Core Programming   
9                               Software Architecture      Core Programming   
10                       Calculus 1A: Differentiation             Core Math   
11                           Calculus 1B: Integration             Core Math   
12  Calculus 1C: Coordinate Systems & Infinite Series             Core Math   
13                   Mathematics for Computer Science             Core Math   
14          The Missing Semester of Your CS Education              CS Tools   
15  Build a Modern Computer from First Principles:...          Core Systems   
16  Build a Modern Computer from First Principles:...          Core Systems   
17               Operating Systems: Three Easy Pieces          Core Systems   
18           Computer Networking: a Top-Down Approach          Core Systems   
19  Divide and Conquer, Sorting and Searching, and...           Core Theory   
20  Graph Search, Shortest Paths, and Data Structures           Core Theory   
21  Greedy Algorithms, Minimum Spanning Trees, and...           Core Theory   
22  Shortest Paths Revisited, NP-Complete Problems...           Core Theory   
23                         Cybersecurity Fundamentals         Core Security   
24                        Principles of Secure Coding         Core Security   
25               Identifying Security Vulnerabilities         Core Security   
26  Identifying Security Vulnerabilities in C/C++ ...         Core Security   
27                     Databases: Modeling and Theory     Core Applications   
28            Databases: Relational Databases and SQL     Core Applications   
29                     Databases: Semistructured Data     Core Applications   
30                                   Machine Learning     Core Applications   
31                                  Computer Graphics     Core Applications   
32                 Software Engineering: Introduction     Core Applications   
33                 Ethics, Technology and Engineering           Core Ethics   
34           Intellectual Property Law in Digital Age           Core Ethics   
35                          Data Privacy Fundamentals           Core Ethics   
36                               Parallel Programming  Advanced Programming   
37                                          Compilers  Advanced Programming   
38                            Introduction to Haskell  Advanced Programming   
39                                  Learn Prolog Now!  Advanced Programming   
40                                 Software Debugging  Advanced Programming   
41                                   Software Testing  Advanced Programming   
42                                      Final Project         Final Project   

       Duration            Effort  Total Hours Lower Bound  Start Date  \
0      10 weeks     10 hours/week                      100  28-07-2022   
1       9 weeks     15 hours/week                      135  01-09-2022   
2       7 weeks   8-10 hours/week                       56  18-10-2022   
3       6 weeks   8-10 hours/week                       48  06-11-2022   
4       5 weeks    4-8 hours/week                       20  23-11-2022   
5       3 weeks    4-8 hours/week                       12  30-11-2022   
6       3 weeks    4-8 hours/week                  

In [52]:
curr

Courses                 Topic  \
0                                Python for Everybody              Intro CS   
1   Introduction to Computer Science and Programmi...              Intro CS   
2                           How to Code - Simple Data      Core Programming   
3                          How to Code - Complex Data      Core Programming   
4                       Programming Languages, Part A      Core Programming   
5                       Programming Languages, Part B      Core Programming   
6                       Programming Languages, Part C      Core Programming   
7                              Object-Oriented Design      Core Programming   
8                                     Design Patterns      Core Programming   
9                               Software Architecture      Core Programming   
10                       Calculus 1A: Differentiation             Core Math   
11                           Calculus 1B: Integration             Core Math   
12  Calculus 1C: Coordinate Systems & Infinite Series             Core Math   
13                   Mathematics for Computer Science             Core Math   
14          The Missing Semester of Your CS Education              CS Tools   
15  Build a Modern Computer from First Principles:...          Core Systems   
16  Build a Modern Computer from First Principles:...          Core Systems   
17               Operating Systems: Three Easy Pieces          Core Systems   
18           Computer Networking: a Top-Down Approach          Core Systems   
19  Divide and Conquer, Sorting and Searching, and...           Core Theory   
20  Graph Search, Shortest Paths, and Data Structures           Core Theory   
21  Greedy Algorithms, Minimum Spanning Trees, and...           Core Theory   
22  Shortest Paths Revisited, NP-Complete Problems...           Core Theory   
23                         Cybersecurity Fundamentals         Core Security   
24                        Principles of Secure Coding         Core Security   
25               Identifying Security Vulnerabilities         Core Security   
26  Identifying Security Vulnerabilities in C/C++ ...         Core Security   
27                     Databases: Modeling and Theory     Core Applications   
28            Databases: Relational Databases and SQL     Core Applications   
29                     Databases: Semistructured Data     Core Applications   
30                                   Machine Learning     Core Applications   
31                                  Computer Graphics     Core Applications   
32                 Software Engineering: Introduction     Core Applications   
33                 Ethics, Technology and Engineering           Core Ethics   
34           Intellectual Property Law in Digital Age           Core Ethics   
35                          Data Privacy Fundamentals           Core Ethics   
36                               Parallel Programming  Advanced Programming   
37                                          Compilers  Advanced Programming   
38                            Introduction to Haskell  Advanced Programming   
39                                  Learn Prolog Now!  Advanced Programming   
40                                 Software Debugging  Advanced Programming   
41                                   Software Testing  Advanced Programming   
42                                      Final Project         Final Project   

       Duration            Effort  Total Hours Lower Bound  Start Date  \
0      10 weeks     10 hours/week                      100  28-07-2022   
1       9 weeks     15 hours/week                      135  01-09-2022   
2       7 weeks   8-10 hours/week                       56  18-10-2022   
3       6 weeks   8-10 hours/week                       48  06-11-2022   
4       5 weeks    4-8 hours/week                       20  23-11-2022   
5       3 weeks    4-8 hours/week                       12  30-11-2022   
6       3 weeks    4-8 hours/week                  

In [31]:
curr.to_csv("../data/curr.csv")

In [53]:
course_codes = pd.read_csv("../data/curr.csv")

In [54]:
course_codes

Unnamed: 0                                            Courses  \
0            0                               Python for Everybody   
1            1  Introduction to Computer Science and Programmi...   
2            2                          How to Code - Simple Data   
3            3                         How to Code - Complex Data   
4            4                      Programming Languages, Part A   
5            5                      Programming Languages, Part B   
6            6                      Programming Languages, Part C   
7            7                             Object-Oriented Design   
8            8                                    Design Patterns   
9            9                              Software Architecture   
10          10                       Calculus 1A: Differentiation   
11          11                           Calculus 1B: Integration   
12          12  Calculus 1C: Coordinate Systems & Infinite Series   
13          13                   Mathematics for Computer Science   
14          14          The Missing Semester of Your CS Education   
15          15  Build a Modern Computer from First Principles:...   
16          16  Build a Modern Computer from First Principles:...   
17          17               Operating Systems: Three Easy Pieces   
18          18           Computer Networking: a Top-Down Approach   
19          19  Divide and Conquer, Sorting and Searching, and...   
20          20  Graph Search, Shortest Paths, and Data Structures   
21          21  Greedy Algorithms, Minimum Spanning Trees, and...   
22          22  Shortest Paths Revisited, NP-Complete Problems...   
23          23                         Cybersecurity Fundamentals   
24          24                        Principles of Secure Coding   
25          25               Identifying Security Vulnerabilities   
26          26  Identifying Security Vulnerabilities in C/C++ ...   
27          27                     Databases: Modeling and Theory   
28          28            Databases: Relational Databases and SQL   
29          29                     Databases: Semistructured Data   
30          30                                   Machine Learning   
31          31                                  Computer Graphics   
32          32                 Software Engineering: Introduction   
33          33                 Ethics, Technology and Engineering   
34          34           Intellectual Property Law in Digital Age   
35          35                          Data Privacy Fundamentals   
36          36                               Parallel Programming   
37          37                                          Compilers   
38          38                            Introduction to Haskell   
39          39                                  Learn Prolog Now!   
40          40                                 Software Debugging   
41          41                                   Software Testing   
42          42                                      Final Project   
43          43                                  C++ For Everybody   
44          44                                 Java For Everybody   
45          45                                     Linear Algebra   
46          46                                High School Algebra   
47          47                                        Probability   
48          48                                           Basic Cs   
49          49                      How To Code (Hear Instructor)   
50          50                                   High School Math   
51          51                                  Scala Programming   
52          52                                      C Programming   

                   Topic     Duration            Effort Course Code  
0               Intro CS     10 weeks     10 hours/week       IC001  
1               Intro CS      9 weeks     15 hours/week       IC002  
2       Core Programming      7 weeks   8-10 hours/week       CP001

In [55]:
curr

Courses                 Topic  \
0                                Python for Everybody              Intro CS   
1   Introduction to Computer Science and Programmi...              Intro CS   
2                           How to Code - Simple Data      Core Programming   
3                          How to Code - Complex Data      Core Programming   
4                       Programming Languages, Part A      Core Programming   
5                       Programming Languages, Part B      Core Programming   
6                       Programming Languages, Part C      Core Programming   
7                              Object-Oriented Design      Core Programming   
8                                     Design Patterns      Core Programming   
9                               Software Architecture      Core Programming   
10                       Calculus 1A: Differentiation             Core Math   
11                           Calculus 1B: Integration             Core Math   
12  Calculus 1C: Coordinate Systems & Infinite Series             Core Math   
13                   Mathematics for Computer Science             Core Math   
14          The Missing Semester of Your CS Education              CS Tools   
15  Build a Modern Computer from First Principles:...          Core Systems   
16  Build a Modern Computer from First Principles:...          Core Systems   
17               Operating Systems: Three Easy Pieces          Core Systems   
18           Computer Networking: a Top-Down Approach          Core Systems   
19  Divide and Conquer, Sorting and Searching, and...           Core Theory   
20  Graph Search, Shortest Paths, and Data Structures           Core Theory   
21  Greedy Algorithms, Minimum Spanning Trees, and...           Core Theory   
22  Shortest Paths Revisited, NP-Complete Problems...           Core Theory   
23                         Cybersecurity Fundamentals         Core Security   
24                        Principles of Secure Coding         Core Security   
25               Identifying Security Vulnerabilities         Core Security   
26  Identifying Security Vulnerabilities in C/C++ ...         Core Security   
27                     Databases: Modeling and Theory     Core Applications   
28            Databases: Relational Databases and SQL     Core Applications   
29                     Databases: Semistructured Data     Core Applications   
30                                   Machine Learning     Core Applications   
31                                  Computer Graphics     Core Applications   
32                 Software Engineering: Introduction     Core Applications   
33                 Ethics, Technology and Engineering           Core Ethics   
34           Intellectual Property Law in Digital Age           Core Ethics   
35                          Data Privacy Fundamentals           Core Ethics   
36                               Parallel Programming  Advanced Programming   
37                                          Compilers  Advanced Programming   
38                            Introduction to Haskell  Advanced Programming   
39                                  Learn Prolog Now!  Advanced Programming   
40                                 Software Debugging  Advanced Programming   
41                                   Software Testing  Advanced Programming   
42                                      Final Project         Final Project   

       Duration            Effort  Total Hours Lower Bound  Start Date  \
0      10 weeks     10 hours/week                      100  28-07-2022   
1       9 weeks     15 hours/week                      135  01-09-2022   
2       7 weeks   8-10 hours/week                       56  18-10-2022   
3       6 weeks   8-10 hours/week                       48  06-11-2022   
4       5 weeks    4-8 hours/week                       20  23-11-2022   
5       3 weeks    4-8 hours/week                       12  30-11-2022   
6       3 weeks    4-8 hours/week                  

In [56]:
curr = curr[["Courses","Prerequisites"]]

In [57]:
def capitalize(s : str):
    return s.upper().strip()

In [58]:
course_codes["Courses"] = course_codes["Courses"].map(capitalize)

In [59]:
def prereq2coursecode(preqs : str):
    if str(type(preqs)) != str(type(0.9)):
        preqs = preqs.split(";")
        a = []
        for i in preqs:
            a.append(course_codes[course_codes["Courses"] == i.upper().strip()]["Course Code"])
        a = np.array(a).flatten()
        return ";".join(a)

In [60]:
curr["Prerequisite Codes"] = curr["Prerequisites"].map(prereq2coursecode)

C:\Users\mitug\AppData\Local\Temp/ipykernel_22264/2520167944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr["Prerequisite Codes"] = curr["Prerequisites"].map(prereq2coursecode)


In [61]:
curr

Courses  \
0                                Python for Everybody   
1   Introduction to Computer Science and Programmi...   
2                           How to Code - Simple Data   
3                          How to Code - Complex Data   
4                       Programming Languages, Part A   
5                       Programming Languages, Part B   
6                       Programming Languages, Part C   
7                              Object-Oriented Design   
8                                     Design Patterns   
9                               Software Architecture   
10                       Calculus 1A: Differentiation   
11                           Calculus 1B: Integration   
12  Calculus 1C: Coordinate Systems & Infinite Series   
13                   Mathematics for Computer Science   
14          The Missing Semester of Your CS Education   
15  Build a Modern Computer from First Principles:...   
16  Build a Modern Computer from First Principles:...   
17               Operating Systems: Three Easy Pieces   
18           Computer Networking: a Top-Down Approach   
19  Divide and Conquer, Sorting and Searching, and...   
20  Graph Search, Shortest Paths, and Data Structures   
21  Greedy Algorithms, Minimum Spanning Trees, and...   
22  Shortest Paths Revisited, NP-Complete Problems...   
23                         Cybersecurity Fundamentals   
24                        Principles of Secure Coding   
25               Identifying Security Vulnerabilities   
26  Identifying Security Vulnerabilities in C/C++ ...   
27                     Databases: Modeling and Theory   
28            Databases: Relational Databases and SQL   
29                     Databases: Semistructured Data   
30                                   Machine Learning   
31                                  Computer Graphics   
32                 Software Engineering: Introduction   
33                 Ethics, Technology and Engineering   
34           Intellectual Property Law in Digital Age   
35                          Data Privacy Fundamentals   
36                               Parallel Programming   
37                                          Compilers   
38                            Introduction to Haskell   
39                                  Learn Prolog Now!   
40                                 Software Debugging   
41                                   Software Testing   
42                                      Final Project   

                                        Prerequisites Prerequisite Codes  
0                                                   -                     
1                                 high school algebra               P004  
2                                                   -                     
3                           How to Code - Simple Data              CP001  
4                       How to Code (Hear instructor)               P007  
5                       Programming Languages, Part A              CP003  
6                       Programming Languages, Part B              CP004  
7                                  Java for Everybody               P002  
8                              Object-Oriented Design              CP006  
9                                     Design Patterns              CP007  
10                                   high school math               P008  
11                       Calculus 1A: Differentiation              CM001  
12                           Calculus 1B: Integration              CM002  
13  Calculus 1C: Coordinate Systems & Infinite Series              CM003  
14                                                  -                     
15                                      C Programming               P010  
16  Build a Modern Computer from First Principles:...             CRS001  
17                                      C Programming               P010  
18              Linear Algebra; Probability; basic CS     P003;P005;P006  
19                   Mathematics for 

In [32]:
curr.to_csv("../data/curriculum_with_prerequisites.csv")

In [33]:
curr = pd.read_csv("../data/curriculum_with_prerequisites.csv")

In [34]:
curr

Unnamed: 0                                            Courses  \
0            0                               Python for Everybody   
1            1  Introduction to Computer Science and Programmi...   
2            2                          How to Code - Simple Data   
3            3                         How to Code - Complex Data   
4            4                      Programming Languages, Part A   
5            5                      Programming Languages, Part B   
6            6                      Programming Languages, Part C   
7            7                             Object-Oriented Design   
8            8                                    Design Patterns   
9            9                              Software Architecture   
10          10                       Calculus 1A: Differentiation   
11          11                           Calculus 1B: Integration   
12          12  Calculus 1C: Coordinate Systems & Infinite Series   
13          13                   Mathematics for Computer Science   
14          14          The Missing Semester of Your CS Education   
15          15  Build a Modern Computer from First Principles:...   
16          16  Build a Modern Computer from First Principles:...   
17          17               Operating Systems: Three Easy Pieces   
18          18           Computer Networking: a Top-Down Approach   
19          19  Divide and Conquer, Sorting and Searching, and...   
20          20  Graph Search, Shortest Paths, and Data Structures   
21          21  Greedy Algorithms, Minimum Spanning Trees, and...   
22          22  Shortest Paths Revisited, NP-Complete Problems...   
23          23                         Cybersecurity Fundamentals   
24          24                        Principles of Secure Coding   
25          25               Identifying Security Vulnerabilities   
26          26  Identifying Security Vulnerabilities in C/C++ ...   
27          27                     Databases: Modeling and Theory   
28          28            Databases: Relational Databases and SQL   
29          29                     Databases: Semistructured Data   
30          30                                   Machine Learning   
31          31                                  Computer Graphics   
32          32                 Software Engineering: Introduction   
33          33                 Ethics, Technology and Engineering   
34          34           Intellectual Property Law in Digital Age   
35          35                          Data Privacy Fundamentals   
36          36                               Parallel Programming   
37          37                                          Compilers   
38          38                            Introduction to Haskell   
39          39                                  Learn Prolog Now!   
40          40                                 Software Debugging   
41          41                                   Software Testing   
42          42                                      Final Project   

                                        Prerequisites Prerequisite Codes  
0                                                   -                NaN  
1                                 high school algebra               P004  
2                                                   -                NaN  
3                           How to Code - Simple Data              CP001  
4                       How to Code (Hear instructor)               P007  
5                       Programming Languages, Part A              CP003  
6                       Programming Languages, Part B              CP004  
7                                  Java for Everybody               P002  
8                              Object-Oriented Design              CP006  
9                                     Design Patterns              CP007  
10                                   high school math               P008  
11                       Calculus 1A: Differentiation              

In [77]:
curr.to_csv("../data/curriculum_with_prerequisites.csv")

In [100]:
y.to_csv("../data/edge_list.csv")

In [7]:
df = pd.read_csv("../data/curr.csv")

df

Unnamed: 0                                            Courses  \
0            0                               Python for Everybody   
1            1  Introduction to Computer Science and Programmi...   
2            2                          How to Code - Simple Data   
3            3                         How to Code - Complex Data   
4            4                      Programming Languages, Part A   
5            5                      Programming Languages, Part B   
6            6                      Programming Languages, Part C   
7            7                             Object-Oriented Design   
8            8                                    Design Patterns   
9            9                              Software Architecture   
10          10                       Calculus 1A: Differentiation   
11          11                           Calculus 1B: Integration   
12          12  Calculus 1C: Coordinate Systems & Infinite Series   
13          13                   Mathematics for Computer Science   
14          14          The Missing Semester of Your CS Education   
15          15  Build a Modern Computer from First Principles:...   
16          16  Build a Modern Computer from First Principles:...   
17          17               Operating Systems: Three Easy Pieces   
18          18           Computer Networking: a Top-Down Approach   
19          19  Divide and Conquer, Sorting and Searching, and...   
20          20  Graph Search, Shortest Paths, and Data Structures   
21          21  Greedy Algorithms, Minimum Spanning Trees, and...   
22          22  Shortest Paths Revisited, NP-Complete Problems...   
23          23                         Cybersecurity Fundamentals   
24          24                        Principles of Secure Coding   
25          25               Identifying Security Vulnerabilities   
26          26  Identifying Security Vulnerabilities in C/C++ ...   
27          27                     Databases: Modeling and Theory   
28          28            Databases: Relational Databases and SQL   
29          29                     Databases: Semistructured Data   
30          30                                   Machine Learning   
31          31                                  Computer Graphics   
32          32                 Software Engineering: Introduction   
33          33                 Ethics, Technology and Engineering   
34          34           Intellectual Property Law in Digital Age   
35          35                          Data Privacy Fundamentals   
36          36                               Parallel Programming   
37          37                                          Compilers   
38          38                            Introduction to Haskell   
39          39                                  Learn Prolog Now!   
40          40                                 Software Debugging   
41          41                                   Software Testing   
42          42                                      Final Project   
43          43                                  C++ For Everybody   
44          44                                 Java For Everybody   
45          45                                     Linear Algebra   
46          46                                High School Algebra   
47          47                                        Probability   
48          48                                           Basic Cs   
49          49                      How To Code (Hear Instructor)   
50          50                                   High School Math   
51          51                                  Scala Programming   
52          52                                      C Programming   

                   Topic     Duration            Effort Course Code  
0               Intro CS     10 weeks     10 hours/week       IC001  
1               Intro CS      9 weeks     15 hours/week       IC002  
2       Core Programming      7 weeks   8-10 hours/week       CP001

In [8]:
x = df.to_dict()

In [9]:
x

{'Unnamed: 0': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49,
  50: 50,
  51: 51,
  52: 52},
 'Courses': {0: 'Python for Everybody',
  1: 'Introduction to Computer Science and Programming using Python',
  2: 'How to Code - Simple Data',
  3: 'How to Code - Complex Data',
  4: 'Programming Languages, Part A',
  5: 'Programming Languages, Part B',
  6: 'Programming Languages, Part C',
  7: 'Object-Oriented Design',
  8: 'Design Patterns',
  9: 'Software Architecture',
  10: 'Calculus 1A: Differentiation',
  11: 'Calculus 1B: Integration',
  12: 'Calculus 1C: Coord

In [10]:
data = dict()

In [11]:
df2 = pd.read_csv("../data/curriculum_with_prerequisites.csv")
y = df2.to_dict()

In [12]:
df2

Unnamed: 0                                            Courses  \
0            0                               Python for Everybody   
1            1  Introduction to Computer Science and Programmi...   
2            2                          How to Code - Simple Data   
3            3                         How to Code - Complex Data   
4            4                      Programming Languages, Part A   
5            5                      Programming Languages, Part B   
6            6                      Programming Languages, Part C   
7            7                             Object-Oriented Design   
8            8                                    Design Patterns   
9            9                              Software Architecture   
10          10                       Calculus 1A: Differentiation   
11          11                           Calculus 1B: Integration   
12          12  Calculus 1C: Coordinate Systems & Infinite Series   
13          13                   Mathematics for Computer Science   
14          14          The Missing Semester of Your CS Education   
15          15  Build a Modern Computer from First Principles:...   
16          16  Build a Modern Computer from First Principles:...   
17          17               Operating Systems: Three Easy Pieces   
18          18           Computer Networking: a Top-Down Approach   
19          19  Divide and Conquer, Sorting and Searching, and...   
20          20  Graph Search, Shortest Paths, and Data Structures   
21          21  Greedy Algorithms, Minimum Spanning Trees, and...   
22          22  Shortest Paths Revisited, NP-Complete Problems...   
23          23                         Cybersecurity Fundamentals   
24          24                        Principles of Secure Coding   
25          25               Identifying Security Vulnerabilities   
26          26  Identifying Security Vulnerabilities in C/C++ ...   
27          27                     Databases: Modeling and Theory   
28          28            Databases: Relational Databases and SQL   
29          29                     Databases: Semistructured Data   
30          30                                   Machine Learning   
31          31                                  Computer Graphics   
32          32                 Software Engineering: Introduction   
33          33                 Ethics, Technology and Engineering   
34          34           Intellectual Property Law in Digital Age   
35          35                          Data Privacy Fundamentals   
36          36                               Parallel Programming   
37          37                                          Compilers   
38          38                            Introduction to Haskell   
39          39                                  Learn Prolog Now!   
40          40                                 Software Debugging   
41          41                                   Software Testing   
42          42                                      Final Project   

                   Topic     Duration            Effort  \
0               Intro CS     10 weeks     10 hours/week   
1               Intro CS      9 weeks     15 hours/week   
2       Core Programming      7 weeks   8-10 hours/week   
3       Core Programming      6 weeks   8-10 hours/week   
4       Core Programming      5 weeks    4-8 hours/week   
5       Core Programming      3 weeks    4-8 hours/week   
6       Core Programming      3 weeks    4-8 hours/week   
7       Core Programming      4 weeks      4 hours/week   
8       Core Programming      4 weeks      4 hours/week   
9       Core Programming      4 weeks    2-5 hours/week   
10             Core Math     13 weeks   6-10 hours/week   
11             Core Math     13 weeks   5-10 hours/week   
12             Core Math      6 weeks   5-10 hours/week   
13             Core Math     13 weeks      5 hours/week   
14              CS Tools      2 weeks     12 hours/week   
15          Core System

In [14]:
for i in x['Course Code']:
      key = x['Course Code'][i]
      data[key] = dict()
      data[key]["Course Name"] = x["Courses"][i]
      d = x["Duration"][i]
      if type(d) != type(3.45):
         if "-" in d:
            data[key]["Duration"] = int(np.mean(list(map(int, d.replace("weeks","").split("-")))))
         else:
            data[key]["Duration"] = int(d.replace("weeks","").strip())
      else:
         data[key]["Duration"] = 30
      data[key]["Prerequisites"] = []
#print(data)

for i in y['Course Code']:
      p = y["Prerequisite Codes"][i]
      #print(y['Course Code'][i], p)
      if type(p) != type(3.45):
            data[y["Course Code"][i]]["Prerequisites"] = p.split(";")

In [37]:
import json
json.dump(data, open("../data/course_data.json", "w"), indent=4)

In [6]:
for i in x['Course Code']:
    if type(x["Duration"][i]) != type(3.4):
        print(x["Course Code"][i], x["Duration"][i].replace("weeks",""))
    else:
        print(x["Course Code"][i], 30)

IC001 10 
IC002 9 
CP001 7 
CP002 6 
CP003 5 
CP004 3 
CP005 3 
CP006 4 
CP007 4 
CP008 4 
CM001 13 
CM002 13 
CM003 6 
CM004 13 
CST001 2 
CRS001 6 
CRS002 6 
CRS003 10-12 
CRS004 8 
CT001 4 
CT002 4 
CT003 4 
CT004 4 
CS001 8 
CS002 4 
CS003 4 
CS004 4 
CA001 2 
CA002 2 
CA003 2 
CA004 11 
CA005 6 
CA006 6 
CE001 9 
CE002 4 
CE003 3 
AP001 4 
AP002 9 
AP003 14 
AP004 12 
AP005 8 
AP006 4 
FP001 30
P001 13 
P002 12 
P003 11 
P004 2 
P005 8 
P006 6 
P007 4 
P008 2 
P009 18 
P010 11 
